In [5]:
# Imports e instalações

import pandas as pd
import numpy as np

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

#!pip install scikit-learn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Métricas
from sklearn.metrics import (accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report)


In [6]:
df_clima = pd.read_csv(r'C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\australia_clima_v4.csv', sep=',')
df_clima

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,ano,mes,dia,trimestre_quarter,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna
0,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,ENE,...,2009,1,1,1,False,False,False,False,False,False
1,2009-01-02,Cobar,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,...,2009,1,2,1,False,False,False,False,False,False
2,2009-01-04,Cobar,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,...,2009,1,4,1,False,False,False,False,False,False
3,2009-01-05,Cobar,21.9,38.4,0.0,11.4,12.2,WNW,31.0,WNW,...,2009,1,5,1,False,False,False,False,False,False
4,2009-01-06,Cobar,24.2,41.0,0.0,11.2,8.4,WNW,35.0,NW,...,2009,1,6,1,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56565,2017-06-20,Darwin,19.3,33.4,0.0,6.0,11.0,ENE,35.0,SE,...,2017,6,20,2,False,False,False,False,False,False
56566,2017-06-21,Darwin,21.2,32.6,0.0,7.6,8.6,E,37.0,SE,...,2017,6,21,2,False,False,False,False,False,False
56567,2017-06-22,Darwin,20.7,32.8,0.0,5.6,11.0,E,33.0,E,...,2017,6,22,2,False,False,False,False,False,False
56568,2017-06-23,Darwin,19.5,31.8,0.0,6.2,10.6,ESE,26.0,SE,...,2017,6,23,2,False,False,False,False,False,False


In [ ]:
# Estatísticas Gerais
df_clima.info()
df_clima.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56570 entries, 0 to 56569
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   data                     56570 non-null  object 
 1   localidade               56570 non-null  object 
 2   temp_min                 56570 non-null  float64
 3   temp_max                 56570 non-null  float64
 4   chuva_mm                 56570 non-null  float64
 5   evaporacao               56570 non-null  float64
 6   horas_sol                56570 non-null  float64
 7   vento_rajada_direcao     56570 non-null  object 
 8   vento_rajada_velocidade  56570 non-null  float64
 9   vento_direcao_9h         56570 non-null  object 
 10  vento_direcao_15h        56570 non-null  object 
 11  vento_veloc_9h           56570 non-null  float64
 12  vento_veloc_15h          56570 non-null  float64
 13  umidade_9h               56570 non-null  float64
 14  umidade_15h           

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,ano,mes,dia,trimestre_quarter,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna
count,56570,56570,56570.000000,56570.000000,56570.000000,56570.000000,56570.000000,56570,56570.000000,56570,...,56570.000000,56570.000000,56570.000000,56570.000000,56570,56570,56570,56570,56570,56570
unique,3416,26,NaN,NaN,NaN,NaN,NaN,16,NaN,16,...,NaN,NaN,NaN,NaN,1,1,1,1,1,1
top,2009-01-04,Darwin,NaN,NaN,NaN,NaN,NaN,E,NaN,N,...,NaN,NaN,NaN,NaN,False,False,False,False,False,False
freq,25,3063,NaN,NaN,NaN,NaN,NaN,4531,NaN,4981,...,NaN,NaN,NaN,NaN,56570,56570,56570,56570,56570,56570
mean,NaN,NaN,13.458626,24.210032,2.131868,5.499295,7.732842,NaN,40.866343,NaN,...,2012.219445,6.423369,15.721584,2.477903,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,6.417872,6.973173,7.014007,3.696565,3.758660,NaN,13.340615,NaN,...,2.450910,3.450963,8.783534,1.121646,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,-6.700000,4.100000,0.000000,0.000000,0.000000,NaN,9.000000,NaN,...,2007.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,8.600000,18.700000,0.000000,2.800000,5.000000,NaN,31.000000,NaN,...,2010.000000,3.000000,8.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,13.200000,23.900000,0.000000,5.000000,8.600000,NaN,39.000000,NaN,...,2012.000000,6.000000,16.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,18.400000,29.700000,0.600000,7.400000,10.700000,NaN,48.000000,NaN,...,2014.000000,9.000000,23.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Valores Nulos
df_clima.isnull().sum()

data                       0
localidade                 0
temp_min                   0
temp_max                   0
chuva_mm                   0
evaporacao                 0
horas_sol                  0
vento_rajada_direcao       0
vento_rajada_velocidade    0
vento_direcao_9h           0
vento_direcao_15h          0
vento_veloc_9h             0
vento_veloc_15h            0
umidade_9h                 0
umidade_15h                0
pressao_9h                 0
pressao_15h                0
nuvem_9h                   0
nuvem_15h                  0
temp_9h                    0
temp_15h                   0
choveu_hoje_fex            0
chove_amanha_vtr           0
ano                        0
mes                        0
dia                        0
trimestre_quarter          0
horas_sol_isna             0
evaporacao_isna            0
nuvem_9h_isna              0
nuvem_15h_isna             0
pressao_9h_isna            0
pressao_15h_isna           0
dtype: int64

In [8]:
df_clima['chove_amanha_vtr'].value_counts(normalize=True)

chove_amanha_vtr
No     0.779477
Yes    0.220523
Name: proportion, dtype: float64

In [14]:
# Normalização da variável alvo
df_clima['chove_amanha_vtr'] = df_clima['chove_amanha_vtr'].map({'Yes': 1, 'No': 0})
df_clima['chove_amanha_vtr']

0        0
1        0
2        0
3        0
4        0
        ..
56565    0
56566    0
56567    0
56568    0
56569    0
Name: chove_amanha_vtr, Length: 56570, dtype: int64

In [27]:
df_clima['chove_amanha_vtr'].value_counts(normalize=True)

chove_amanha_vtr
0    0.779477
1    0.220523
Name: proportion, dtype: float64

In [ ]:
# Início do teste com modelo de Regressão Logística

# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

# Regressão logística é um modelo de classificação usado para prever a probabilidade de uma variável dependente categórica com base em uma ou mais variáveis independentes. É amplamente utilizado em problemas de classificação binária, onde a variável dependente tem dois resultados possíveis (por exemplo, sim/não, verdadeiro/falso).
# A regressão logística estima a probabilidade de um evento ocorrer, modelando a relação entre as variáveis independentes e a variável dependente usando a função logística (sigmóide). A saída do modelo é uma probabilidade que pode ser convertida em uma classe binária com base em um limiar predefinido (geralmente 0,5).
# Em regressão, ela observa todas as variáveis do dia, efetua um peso para cada uma delas, soma tudo e aplica uma função sigmóide para transformar o resultado em uma probabilidade entre 0 e 1.
# Se a probabilidade for maior que 0,5, o modelo prevê a classe "1" (evento ocorre); caso contrário, prevê a classe "0" (evento não ocorre).
# Exemplo:
# umidade = peso +2.0
# pressao = peso -1.5
# horas de sol = peso -3.0
# Todas essas variáveis são multiplicadas pelos seus respectivos pesos, somadas e aplicadas na função sigmóide para obter a probabilidade final // Chuva ou Não Chuva
# Probabilidade = sigmoid(w1*x1 + w2*x2 + ... + wn*xn + b)
# Onde: w1, w2, ..., wn são os pesos (coeficientes) das variáveis independentes
# x1, x2, ..., xn são as variáveis independentes
# b é o termo de interceptação (bias)
# Sigmoid entra a função matemática que transforma o resultado em uma probabilidade entre 0 e 1.
# O modelo passa os números pela função sigmóide para gerar uma probabilidade, que é então convertida em uma classe binária com base em um limiar predefinido (geralmente 0,5).
# A fórmula matemática da função sigmóide é:
# sigmoid(z) = 1 / (1 + e^(-z))
# Onde "e" é a base do logaritmo natural (aproximadamente 2,71828) e "z" é o valor resultante da soma ponderada das variáveis independentes mais o termo de interceptação.
# z (resultado da soma) // muito negativo // 0 // muito positivo || saída da sigmóide // 0 // 0,5 // 1
# O resultado é algo como 'probabilidade de chover amanhã é 0,8 (80%)', e se for maior que 0,5, o modelo prevê que vai chover (classe 1); caso contrário, prevê que não vai chover (classe 0).
# Esse modelo aprende os pesos (coeficientes) das variáveis independentes durante o treinamento, ajustando-os para minimizar a diferença entre as previsões do modelo e os resultados reais observados nos dados de treinamento.


In [ ]:
# Definição de X e y
# y = resposta(chove amanhã)
# X = var preditora

X = df_clima.drop(columns=['chove_amanha_vtr'])
y = df_clima['chove_amanha_vtr']

X.shape, y.shape


((56570, 32), (56570,))

In [31]:
display(X.head(3)),display(y.head(3))

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,ano,mes,dia,trimestre_quarter,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna
0,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,ENE,...,2009,1,1,1,False,False,False,False,False,False
1,2009-01-02,Cobar,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,...,2009,1,2,1,False,False,False,False,False,False
2,2009-01-04,Cobar,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,...,2009,1,4,1,False,False,False,False,False,False


0    0
1    0
2    0
Name: chove_amanha_vtr, dtype: int64

(None, None)

In [ ]:
# Corte temporal dos dados para treino e teste

# split_point recebe o índice que separa os dados de treino (70%) dos dados de teste (30%)
split_point = int(len(X) * 0.7)

# X_train recebe os dados preditores de treino (70%)
# X_test recebe os dados preditores de teste (30%)
X_train = X.iloc[:split_point]
X_test  = X.iloc[split_point:]

# y_train recebe os dados alvo de treino (70%)
# y_test recebe os dados alvo de teste (30%)
y_train = y.iloc[:split_point]
y_test  = y.iloc[split_point:]

# Separação de tipo de variáveis (numéricas e categóricas)
# variaveis_numericas - armazene na var colunas com dados numéricos (int e float)
variaveis_numericas = X.select_dtypes(include=['int64', 'float64']).columns
# variaveis_categoricas - armazene na var colunas com dados categóricos (object e category)
variaveis_categoricas = X.select_dtypes(include=['object', 'category']).columns


In [ ]:
# Pipeline numérico

# pipeline_numerico - crie um pipeline para dados numéricos que primeiro preenche valores nulos com a mediana da coluna (SimpleImputer) e depois padroniza os dados (StandardScaler)
pipeline_numerico = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),('scaler', StandardScaler())])

# Pipeline categórico
# pipeline_categorico - crie um pipeline para dados categóricos que primeiro preenche valores nulos com a moda da coluna (SimpleImputer) e depois aplica codificação one-hot (OneHotEncoder)
pipeline_categorico = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),('encoder', OneHotEncoder(handle_unknown='ignore'))])

# Pré processamento
# preprocessador - crie um ColumnTransformer que aplique o pipeline_numerico às variáveis numéricas e o pipeline_categorico às variáveis categóricas
preprocessador = ColumnTransformer(transformers=[('num', pipeline_numerico, variaveis_numericas),('cat', pipeline_categorico, variaveis_categoricas)])

# Treino Reg Logística
# modelo_logistico - crie um pipeline que primeiro aplique o preprocessador e depois treine um modelo de Regressão Logística (LogisticRegression) com max_iter=10000
modelo_logistico = Pipeline(steps=[('preprocessamento', preprocessador),('modelo', LogisticRegression(max_iter=10000))])
modelo_logistico.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessamento', ...), ('modelo', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

In [46]:
# Prev
y_pred = modelo_logistico.predict(X_test)
accuracy_score(y_test, y_pred)


0.8758470331742384

In [49]:
# Matriz de confusão
confusion_matrix(y_test, y_pred)

array([[13139,   684],
       [ 1423,  1725]])

In [54]:
"""
[[TN FP]
 [FN TP]]
TN = acertou “não chove”
FP = falso alarme de chuva
FN = disse “não chove”, mas choveu (erro mais grave)
TP = acertou a chuva
"""

'\n[[TN FP]\n [FN TP]]\nTN = acertou “não chove”\nFP = falso alarme de chuva\nFN = disse “não chove”, mas choveu (erro mais grave)\nTP = acertou a chuva\n'

In [53]:
# Avaliação usando TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
accuracies = []
recalls_chuva = []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    modelo_logistico.fit(X_train, y_train)
    y_pred = modelo_logistico.predict(X_test)

    accuracies.append(accuracy_score(y_test, y_pred))
    recalls_chuva.append(recall_score(y_test, y_pred))

print(f"Acurácia: {np.mean(accuracies):.2f}")
print(f"Recall - chuva: {np.mean(recalls_chuva):.2f}")

Acurácia: 0.86
Recall - chuva: 0.55
